* https://docs.ragas.io/en/latest/concepts/testset_generation.html
* https://github.com/explodinggradients/ragas/blob/main/src/ragas/testset/generator.py

In [1]:
import ragas

/Users/I748920/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/I748920/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from import ollama
from langchain_community.embeddings import HuggingFaceEmbeddings

In [28]:
from langchain_ollama import ChatOllama

llm_model1 = ChatOllama(
    model="llama3.1",
    temperature=0 # increase temp for more creative answers
)

llm_model2= ChatOllama(
    model="llama3.1",
    temperature=0 # increase temp for more creative answers
)

embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name, 
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [29]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# # generator with openai models
# generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
# critic_llm = ChatOpenAI(model="gpt-4")
# embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm=llm_model1,
    critic_llm=llm_model2,
    embeddings=embedding_model
)

In [31]:
generator

TestsetGenerator(generator_llm=LangchainLLMWrapper(run_config=RunConfig(timeout=180, max_retries=10, max_wait=60, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42)), critic_llm=LangchainLLMWrapper(run_config=RunConfig(timeout=180, max_retries=10, max_wait=60, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42)), embeddings=<ragas.embeddings.base.LangchainEmbeddingsWrapper object at 0x16f8ce550>, docstore=InMemoryDocumentStore(splitter=<langchain_text_splitters.base.TokenTextSplitter object at 0x16f8ce580>, nodes=[], node_embeddings_list=[], node_map={}, run_config=RunConfig(timeout=180, max_retries=10, max_wait=60, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42)))

In [36]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_pdfs(file_paths):
    """
    file_paths must end with .pdf
    PyPDFLoader auto splits the pdf into pages, each page is 1 Document object split by page number
    note that the splitting by page number is not perfect, the actual page number might be +/- 1-2pages.

    returns a dict of key: file_path and value: list of document objects
    """
    documents_dict = {}   
    for f in tqdm(file_paths):
        loader = PyPDFLoader(file_path = f)
        documents = loader.load()
        documents_dict[f] = documents
    return documents_dict

def chunk_list_of_documents(documents):
    """
    input a list of documents as Document objects

    output a list of chunks as Document objects
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 100, # using 20% is a good start
        length_function=len,
        is_separator_regex=False,
        add_start_index=True
    )

    chunks = text_splitter.split_documents(documents)    
    return chunks

In [82]:
import os
import glob
from tqdm import tqdm

# folder_path = "/Users/i748920/Desktop/llms-learning/pdf-chatbot-app/data/short-elements-of-statistical-learning-book"
# os.path.exists(folder_path)
# file_paths = glob.glob(f"{folder_path}/*.pdf")
documents_dict = load_pdfs(
    file_paths=[
        "/Users/i748920/Desktop/llms-learning/pdf-chatbot-app/data/short-elements-of-statistical-learning-book/chap18.pdf"
    ]
)

100%|██████████| 1/1 [00:04<00:00,  4.31s/it]


In [84]:
len(list(documents_dict.values()))

1

In [86]:
docs = []
for lst in list(documents_dict.values()):
    docs.extend(lst)

len(docs)

50

In [88]:
# generate testset
testset = generator.generate_with_langchain_docs(
    documents=docs, 
    test_size=10, 
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25}
)
#102 because of the 50 docs being split into chunk_size: int = 1024

Filename and doc_id are the same for all nodes.                   
Generating:  60%|██████    | 6/10 [57:38<38:25, 576.45s/it]


ZeroDivisionError: division by zero

max retries exceeded for SimpleEvolution(generator_llm=LangchainLLMWrapper(run_config=RunConfig(timeout=180, max_retries=15, max_wait=90, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42)), docstore=InMemoryDocumentStore(splitter=<langchain_text_splitters.base.TokenTextSplitter object at 0x16f8ce580>, nodes=[Node(metadata={'source': '/Users/i748920/Desktop/llms-learning/pdf-chatbot-app/data/short-elements-of-statistical-learning-book/chap18.pdf', 'page': 0}, page_content='This is page 649\nPrinter: Opaque this\n18\nHigh-Dimensional Problems: p≫N\n18.1 When pis Much Bigger than N\nIn this chapter we discuss prediction problems in which the n umber of\nfeaturespis much larger than the number of observations N, often written\np≫N. Such problems have become of increasing importance, espec ially in\ngenomics and other areas of computational biology. We will s ee that high\nvariance and overﬁtting are a major concern in this setting. As a result,\nsimple, h

In [ ]:
1

In [90]:
testset

NameError: name 'testset' is not defined

In [ ]:
test_df = testset.to_pandas()
test_df